In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd

df_train = pd.read_csv('train_kaggle.csv')
df_train

,Id,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,1
9,9,0


In [3]:
import pandas as pd
def load_features(col):
    return pd.read_parquet("filtered_features/filtered_feature{}.gzip".format(col))

In [4]:
import pandas as pd

def load_test_features(col):
    return pd.read_parquet("filtered_test_features/filtered_test_feature{}.gzip".format(col))

In [5]:
NUM_OF_COL = 40

In [23]:
SkipCol = {}
# SkipCol[18] = True
SkipCol

{}

In [24]:
y = df_train['label']
y = y.values
y

array([0, 0, 0, ..., 1, 0, 1], dtype=int64)

In [25]:
from sklearn import decomposition
import numpy as np

XTrainAll = []
for col in range(NUM_OF_COL):
    if col in SkipCol:
        continue
    tf = load_features(col)
    XTrainAll.append(tf)
XTrainAll = np.concatenate(XTrainAll,  axis=1)
XTrainAll.shape

(30336, 14564)

In [26]:
XTestAll = []
for col in range(NUM_OF_COL):
    if col in SkipCol:
        continue
    tf = load_test_features(col)
    tf = tf.replace([np.inf], np.finfo('float32').max).replace([np.inf, -np.inf], np.finfo('float32').min).fillna(0)
    XTestAll.append(tf)
XTestAll = np.concatenate(XTestAll,  axis=1)
XTestAll.shape

(10000, 14564)

In [27]:
df_test = pd.read_csv('sample_solution.csv')


In [28]:
y.shape

(30336,)

In [29]:
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

clf = ExtraTreesClassifier(n_estimators=50)
# clf = GradientBoostingClassifier()
clf = clf.fit(XTrainAll, y)
modelSelection = SelectFromModel(clf, prefit=True, max_features=8000)
XTrain = modelSelection.transform(XTrainAll)
XTrain.shape

(30336, 3372)

In [30]:
'''
from sklearn import decomposition

pca = decomposition.PCA(n_components=800)
pca.fit(XTrainAll)
XTrain = pca.transform(XTrainAll)
XTrain.shape
'''
# XTrain = XTrainAll
# XTrain.shape

'\nfrom sklearn import decomposition\n\npca = decomposition.PCA(n_components=800)\npca.fit(XTrainAll)\nXTrain = pca.transform(XTrainAll)\nXTrain.shape\n'

In [31]:
XTest = modelSelection.transform(XTestAll)
XTest.shape

(10000, 3372)

In [32]:
'''
XTest = pca.transform(XTestAll)
XTest.shape
'''
# XTest = XTestAll
# XTest.shape

'\nXTest = pca.transform(XTestAll)\nXTest.shape\n'

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

X_train, X_val, y_train, y_val = train_test_split(XTrain, y, test_size=0.2, random_state=42)
gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train, y_train)
probs = gb_clf.predict_proba(X_val)
y_pred = probs[:, 0]
metrics.roc_auc_score(y_val, y_pred)

In [ ]:
gb_clf = GradientBoostingClassifier()
gb_clf.fit(XTrain, y)

In [ ]:
probs = gb_clf.predict_proba(XTest)
YTest = probs[:, 0]
YTest

In [ ]:
np.max(YTest)

In [ ]:
df_test['Predicted'] = YTest
df_test

In [ ]:
df_test.to_csv('test.csv', index=False)